## Large Language Models, the Chat Format and Tokens
##### Setup
Load the API key and relevant Python libaries.

In [17]:
#!pip install openai
#!pip install tiktoken
#!pip install dotenv

In [1]:
import os
import openai
import tiktoken
from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv())  # read local .env file

openai.api_key = os.environ["OPENAI_API_KEY"]

**Helper function**

In [2]:
def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0,
    )
    return response.choices[0].message["content"]

##### Prompt the model and get completion

In [4]:
response = get_completion("What is the capital of France?")
print(response)

The capital of France is Paris.


##### Tokens

In [5]:
response = get_completion("Take the letters in lollipop and reverse them")
print(response)

The reversed letters of "lollipop" are "pillipol".


In [6]:
# that was incorrect. What about this?
response = get_completion(
    """Take the letters in \
l-o-l-l-i-p-o-p and reverse them"""
)
print(response)

p-o-p-i-l-l-o-l


##### Helper function (chat format)¶
Here's the helper function we'll use in the following cells

In [7]:
def get_completion_from_messages(
    messages,
    model="gpt-3.5-turbo",
    temperature=0,
    max_tokens=500,
):
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature,  # this is the degree of randomness of the model's output
        max_tokens=max_tokens,  # the maximum number of tokens the model can ouptut
    )
    return response.choices[0].message["content"]

In [8]:
messages = [
    {
        "role": "system",
        "content": """You are an assistant who responds in the style of Dr Seuss.""",
    },
    {"role": "user", "content": """write me a very short poem about a happy carrot"""},
]
response = get_completion_from_messages(messages, temperature=1)
print(response)

In a garden so cheery, a carrot did grow,
With a smile on its face, it wanted to show,
It stretched its orange self, up high in the air,
So proud to be tasty, and oh-so rare.

With friends all around, it danced and did sing,
A happy little carrot, a joy it would bring,
It giggled with glee, as it wiggled its root,
Bringing laughter and smiles, from harvest to boot.

With sunshine above and dirt at its feet,
This carrot was happy, so sweet and so neat,
It brightened the garden, it brightened the day,
A happy little carrot, forever at play.

So remember this tale, whenever you see,
A carrot so happy, just like you and me,
With joy in our hearts, and a smile so bright,
We'll make the world happier, with all of our might!


In [9]:
# length
messages = [
    {"role": "system", "content": "All your responses must be one sentence long."},
    {"role": "user", "content": "write me a story about a happy carrot"},
]
response = get_completion_from_messages(messages, temperature=1)
print(response)

Once upon a time, there was a cheerful carrot named Carl who lived in a sunny vegetable garden.


In [10]:
# combined
messages = [
    {
        "role": "system",
        "content": """You are an assistant who responds in the style of Dr Seuss. \
All your responses must be one sentence long.""",
    },
    {"role": "user", "content": """write me a story about a happy carrot"""},
]
response = get_completion_from_messages(messages, temperature=1)
print(response)

Once upon a time in a garden so bright, there was a happy carrot, oh what a delight!


In [12]:
def get_completion_and_token_count(
    messages,
    model="gpt-3.5-turbo",
    temperature=0,
    max_tokens=500,
):
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature,
        max_tokens=max_tokens,
    )

    content = response.choices[0].message["content"]

    token_dict = {
        "prompt_tokens": response["usage"]["prompt_tokens"],
        "completion_tokens": response["usage"]["completion_tokens"],
        "total_tokens": response["usage"]["total_tokens"],
    }

    return content, token_dict, response

In [13]:
messages = [
    {
        "role": "system",
        "content": """You are an assistant who responds in the style of Dr Seuss.""",
    },
    {"role": "user", "content": """write me a very short poem  about a happy carrot"""},
]
response, token_dict, content = get_completion_and_token_count(messages)

In [14]:
print(response)

Oh, the happy carrot, so bright and orange,
With a smile so wide, it's hard to manage.
In the garden it grew, with love and care,
Bathing in sunshine, breathing in fresh air.

Its roots dug deep, in the soil so fine,
Growing tall and strong, like a vine.
With every day that passed, it grew so sweet,
A tasty treat, for all to eat.

From the ground it was plucked, with a joyful cheer,
For the happy carrot had nothing to fear.
It brought smiles to faces, with its crunch and taste,
A veggie delight, not a moment to waste.

So let's celebrate the happy carrot, so grand,
A vibrant veggie, grown by nature's hand.
With every bite, a burst of joy and glee,
Oh, happy carrot, you make us so happy!


In [15]:
print(token_dict)

{'prompt_tokens': 36, 'completion_tokens': 174, 'total_tokens': 210}


In [16]:
print(content)  # the entire original response from LLM

{
  "id": "chatcmpl-8EG071dV7auLbriG6e4X9HLbomxpP",
  "object": "chat.completion",
  "created": 1698408791,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "Oh, the happy carrot, so bright and orange,\nWith a smile so wide, it's hard to manage.\nIn the garden it grew, with love and care,\nBathing in sunshine, breathing in fresh air.\n\nIts roots dug deep, in the soil so fine,\nGrowing tall and strong, like a vine.\nWith every day that passed, it grew so sweet,\nA tasty treat, for all to eat.\n\nFrom the ground it was plucked, with a joyful cheer,\nFor the happy carrot had nothing to fear.\nIt brought smiles to faces, with its crunch and taste,\nA veggie delight, not a moment to waste.\n\nSo let's celebrate the happy carrot, so grand,\nA vibrant veggie, grown by nature's hand.\nWith every bite, a burst of joy and glee,\nOh, happy carrot, you make us so happy!"
      },
      "finish_reason": "stop